<img src="fifa.jpeg" style="float:left">

# Fifa 19 Dataset - Transfer Value predicting Regression Model

# Importing required Packages

In [1]:

import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from mosaicml import *
from mosaicml.constants import MLModelFlavours

import warnings
warnings.filterwarnings('ignore')
pd.read_csv()

/opt/conda/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


TypeError: read_csv() missing 1 required positional argument: 'filepath_or_buffer'

# Loading Dataset

In [2]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/data/fifa.csv'

# Seperating Dataset into Training and Test

In [3]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(ylog, bins='auto')
plt.title("ln(value_eur)")
plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

NameError: name 'data' is not defined

# Training the Model

In [4]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

# Defining a Score Function

In [5]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data_list = payload
    data_array = np.asarray(data_list)
    try:
        prediction = model.predict(data_array)
    except:
        prediction = model.predict(data_array.reshape(1, -1))
    return prediction.tolist()


# Testing Score Function

In [6]:
# to verify if score func works

import requests
req = requests.Request()
req.json = {"payload":X_test}

'''
Reference Sample Payload 

req.json = {"payload": [ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9,
        10,  79,  82]}
        
'''
score(gbm_default, req)

[7371874.6616814695,
 4738569.183719676,
 21420646.724212054,
 18565368.234815326,
 2521068.456590733,
 4927893.686116418,
 17717792.640952606,
 536649.2767679257,
 5379405.813943996,
 2782661.6857576417,
 3109098.8043037425,
 3854078.9689723533,
 4098721.556383511,
 7769807.652539338,
 4692304.3720488595,
 16930281.548238758,
 3473019.6617231523,
 6661265.394274316,
 3016063.3275525416,
 10421430.298660973,
 1978920.712208259,
 2802425.7801835826,
 905938.3801071722,
 7703026.231638116,
 4737163.454637381,
 9200545.134584034,
 40671774.99537543,
 11732600.005104974,
 11769365.628129425,
 127060.88757149465,
 5192798.5972583555,
 3933793.1665185424,
 2461772.7940336023,
 17703016.39503423,
 5660456.160853369,
 2502423.505083097,
 6259869.183645455,
 8698457.579092506,
 1833849.8102294619,
 4711891.257815068,
 2820859.9746349454,
 2385315.8941515293,
 1986428.0458969302,
 7220836.170301249,
 2028027.9431738025,
 1953495.1905439012,
 3606521.7807429824,
 2197176.3340813695,
 6941085.8841

# Calculating Y_true and Y_pred

In [7]:
Y_true = y_test

Y_pred = gbm_default.predict(X_test)
features = list(X_train)

# Registering the Model

In [9]:
register_model(
    gbm_default,
    score,
    "fifa_model",
    "fifa_regression",
    MLModelFlavours.sklearn,
    init_script="mkdir abc \\n cd abc \\n touch abc.txt",
    input_type="json",
    explain_ai=True,
    x_train=X_train, 
    y_train=y_train, 
    y_true=Y_true,
    y_pred=Y_pred,
    feature_names=features,
    feature_ids=features,
    target_names=["Value"],
    model_type="regression"
)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


2021-06-03 07:21:59,854 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progressbar=False
2021-06-03 07:21:59,856 - skater.core.explanations - WARNING - Multiprocessing failed, going single process


[25/25] grid cells ████████████████████ Time elapsed: 1 seconds

2021-06-03 07:22:02,181 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progressbar=False
2021-06-03 07:22:02,184 - skater.core.explanations - WARNING - Multiprocessing failed, going single process


[22/22] grid cells ████████████████████ Time elapsed: 0 seconds

...

[26/26] grid cells ████████████████████ Time elapsed: 1 seconds

2021-06-03 07:22:35,464 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progress_bar=False
2021-06-03 07:22:35,467 - skater.core.explanations - WARNING - Multiprocessing failed, going single process


[16/16] features ████████████████████ Time elapsed: 3 seconds

2021-06-03 07:22:39,852 - skater.core.global_interpretation.tree_surrogate - INFO - post pruning applied ...
2021-06-03 07:22:39,877 - skater.core.global_interpretation.tree_surrogate - INFO - Scorer used MAE
2021-06-03 07:22:39,879 - skater.core.global_interpretation.tree_surrogate - INFO - original score using base model 516013.7860831824
2021-06-03 07:22:39,993 - skater.core.global_interpretation.tree_surrogate - INFO - Summary: childrens of the following nodes are removed []
2021-06-03 07:22:39,996 - skater.core.global_interpretation.tree_surrogate - INFO - Done generating prediction using the surrogate, shape (3750,)
2021-06-03 07:22:39,998 - skater.core.global_interpretation.tree_surrogate - INFO - Done scoring, surrogate score 516013.786; oracle score 324673.818


{'created_by': 'mosaic@sc.com', 'created_on': '2021-06-03T07:21:51.690688+00:00', 'description': 'fifa_regression', 'flavour': 'sklearn', 'id': '5bf5b4c0-fe43-4da8-98de-48466efaa477', 'last_modified_by': 'mosaic@sc.com', 'last_modified_on': '2021-06-03T07:21:51.690702+00:00', 'model_display': True, 'name': 'fifa_model', 'project_id': '173f130c-2439-4824-8435-9df5526d9c02', 'source': '', 'tags': None, 'type': 'model', 'versions': [{'created_by': 'mosaic@sc.com', 'created_on': '2021-06-03T07:21:51.734607+00:00', 'datasource_name': '', 'dependent_model': None, 'deploy_info': None, 'deployments': [], 'description': None, 'docker_image_url': 'registry.lti-aiq.in:443/mosaic-ai-logistics/mosaic-ai-serving:1.0.0-12042021', 'gpu_docker_image_url': 'registry.lti-aiq.in:443/mosaic-ai-logistics/mosaic-ai-serving:gpu-1.0.0-07122021', 'id': '02e68b78-18a9-4d08-9ca4-26a1619046dc', 'init_script': '"pip install --user -i https://pypi.org/simple absl-py==0.12.0\\n pip install --user -i https://pypi.org/

In [ ]:
for i in range(10000):
    import time
    time.sleep(3)
    !echo f'{i}' > ab.csv
    print(i)